In [ ]:
import os

def read_xyz_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        n_atoms = int(lines[0])
        coordinates = []
        for line in lines[2:]:
            split_line = line.split()
            symbol, x, y, z = split_line[0], float(split_line[1]), float(split_line[2]), float(split_line[3])
            coordinates.append([symbol, x, y, z])
        return coordinates

In [ ]:

def generate_gaussian_input(file_name, coordinates, frozen_atoms, dir_path, subdirectory_path):
    with open(os.path.join(subdirectory_path, file_name + '.gjf'), 'w') as f:
        f.write(f"%nprocshared=32\n%mem=96GB\n%chk={file_name}.chk\n")
        f.write(f"#p M062X/def2SVP opt=modredundant geom=connectivity empiricaldispersion=gd3 scrf=(smd,solvent=dichloromethane) NoSymm\n\n")
        f.write(f"{file_name}\n\n")
        f.write("1 2\n")
        for i, coord in enumerate(coordinates):
            f.write(str(coord[0]) + " ")
            if i in frozen_atoms:
                f.write(" -1 ")
            else:
                f.write(" 0 ")
            symbol, x, y, z = coord[0], coord[1], coord[2], coord[3]
            f.write(f"{x:.6f} {y:.6f} {z:.6f}\n")
        f.write(f"\n")

In [ ]:

def generate_batch_file(file_name, subdirectory_path):
    with open(os.path.join(subdirectory_path, 'run.sh'), 'w') as f:
        f.write(f'#!/bin/bash\n#SBATCH --job-name=gaussian_run\n')
        f.write(f'#SBATCH --mem-per-cpu=3000M\n')
        f.write(f'#SBATCH --cpus-per-task=32\n')
        f.write(f'#SBATCH --ntasks=1\n')
        f.write(f'#SBATCH --time=00-24:00\n')
        f.write(f'module load gaussian/g16.c01\n')
        f.write(f'g16 < {file_name}.gjf >& {file_name}.log\n')
        f.write(f'formchk {file_name}.chk {file_name}.fchk\n')

In [ ]:

def main():
    frozen_atoms = list(range(0, 0))
    cwd = os.getcwd()
    for xyz_file in os.listdir(cwd):
        if xyz_file.endswith('.xyz'):
            file_name = os.path.splitext(xyz_file)[0]
            subdirectory_path = ''
            if '_' in file_name:
                directory_name, subdirectory_name = file_name.split('_')[:-1], file_name.split('_')[-1]
                subdirectory_path = os.path.join(*directory_name, subdirectory_name)
            else:
                subdirectory_name = file_name
            dir_path = os.path.join(cwd, subdirectory_name)
            if not os.path.exists(dir_path):
                os.makedirs(dir_path)
            generate_gaussian_input(file_name, read_xyz_file(os.path.join(cwd, xyz_file)), frozen_atoms, cwd, dir_path)
            generate_batch_file(file_name, dir_path)

In [ ]:

if __name__ == "__main__":
    main()
